# Räuber Beute Modell

In [4]:
import numpy as np

In [5]:
METABOLISM = 1
FOOD_MIN_INIT = 1
FOOD_MAX_INIT = 5

In [6]:
class Agent:
    def __init__(self, env, x, y, energy):
        self._env = env
        self._x = x
        self._y = y
        self._energy = energy
        
    def move(self, dx, dy):
        if self.energy <= 0:
            return
        
        self._x += dx
        self._y += dy
        
        if self._x >= self._env.X_MAX:
            self._x = 0
        elif self._x < 0:
            self._x = self._env.X_MAX-1
            
        if self._y >= self._env.Y_MAX:
            self._y = 0
        elif self._y < 0:
            self._y = self._env.Y_MAX-1
        
        self._energy -= METABOLISM
        
    def find_agent(self, agents):
        for a in agents:
            if self.x == a.x and self.y == a.y:
                return a
        return None
    
    def _breed(self, other, agents):
        if not other is None:
            energy_breed = np.round((self.energy + other.energy) / 2)
            offspring = Prey(self.x, self.y, energy_breed)
            agents.append(offspring)
          
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    @property
    def energy(self):
        return self._energy

In [7]:
class Prey(Agent):
    def eat(self, amount):
        self._energy += amount
        
    def get_eaten(self):
        self._energy = 0
        
    def breed(self):
        p = self.find_agent(self._env.prey)
        self._breed(p, self._env.prey)

In [8]:
class Predator(Agent):
    def eat(self):
        p = self.find_agent(self._env.prey)
        if not p is None:
            self._energy += p.energy
            p.get_eaten()
        
    def breed(self):
        p = self.find_agent(self._env.predators)
        self._breed(p, self._env.predators)

In [9]:
class Environment:
    def __init__(self, dims, n_predators, n_prey):
        x_max, y_max = dims
        self.X_MAX = x_max
        self.Y_MAX = y_max
        
        self.predators = [
            Predator(
                self,
                np.random.randint(0, x_max),
                np.random.randint(0, y_max),
                np.random.randint(FOOD_MIN_INIT, FOOD_MAX_INIT)
                ) 
            for _ in range(n_predators)
            ]  
        
        self.prey = [
            Prey(
                self,
                np.random.randint(0, x_max),
                np.random.randint(0, y_max),
                np.random.randint(FOOD_MIN_INIT, FOOD_MAX_INIT)
                ) 
            for _ in range(n_prey)
            ]
    
    def tick(self):
        # breed
        for p in self.prey + self.predators:
            p.breed()
            
        # move
        for p in self.prey + self.predators:
            dx = np.random.randint(0,3)-1
            dy = np.random.randint(0,3)-1
            
            p.move(dx, dy)
        
        # feed
        for p in self.prey:
            p.eat(np.random.randint(0,3))
            
        for p in self.predators:
            p.eat()
            
        # check alive
        for p in self.prey:
            if p.energy <= 0:
                self.prey.remove(p)
        
        for p in self.predators:
            if p.energy <= 0:
                self.predators.remove(p)
    